In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import geopandas as gpd
import contextily as ctx
from matplotlib.colors import LinearSegmentedColormap
import os

# Create visualizations folder if it doesn't exist
output_dir = 'visualizations'
os.makedirs(output_dir, exist_ok=True)

# Load the CSV data
csv_path = "data/aggregated/school_quality_ratings.csv"
df = pd.read_csv(csv_path)

# --- Load and Inspect GeoJSON ---
geojson_path = "data/shapefiles/ga_georgia_zip_codes_geo.min.json"
gdf = gpd.read_file(geojson_path)

# Debug: Print GeoJSON columns to verify ZIP code property
print("GeoJSON columns:", gdf.columns.tolist())

# Assume ZIP code column is 'ZCTA5CE10'; adjust if different
zip_column = 'ZCTA5CE10'  # Update to 'GEOID', 'ZCTA5CE20', etc., if needed

# Filter for ZIP codes in the Atlanta area (based on CSV data)
atlanta_zips = df['ZIP Code'].astype(str).unique()
gdf = gdf[gdf[zip_column].isin(atlanta_zips)]

# Reproject to Web Mercator for basemap compatibility
gdf = gdf.to_crs(epsg=3857)

# --- Visualization 1: GeoPandas Choropleth Map of ZIP Codes by Average Quality Score ---
# Calculate average Quality Score per ZIP code
zip_scores = df.groupby('ZIP Code')['Quality Score'].mean().reset_index()
zip_scores['ZIP Code'] = zip_scores['ZIP Code'].astype(str)

# Merge with geodata
gdf_quality = gdf.merge(zip_scores, left_on=zip_column, right_on='ZIP Code', how='left')

# Plot Quality Score choropleth with basemap
fig, ax = plt.subplots(figsize=(12, 8))
gdf_quality.plot(column='Quality Score', cmap='YlOrRd', legend=True, ax=ax,
                 missing_kwds={'color': 'lightgrey', 'label': 'No Data'},
                 legend_kwds={'label': "Average Quality Score", 'orientation': "horizontal"},
                 edgecolor='black', linewidth=1.5, alpha=0.7)
ctx.add_basemap(ax, crs=gdf_quality.crs, source=ctx.providers.OpenStreetMap.Mapnik)
ax.set_title('Average School Quality Score by ZIP Code in Atlanta Area', fontsize=14)
ax.set_axis_off()
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'zip_code_quality_score_map.png'), dpi=300, bbox_inches='tight')
plt.close()

# --- Visualization 2: GeoPandas Choropleth Map of ZIP Codes by Highest Letter Grade ---
# Define grade order (highest to lowest)
grade_order = ['A', 'B', 'C', 'D', 'F']
df['Letter Grade'] = pd.Categorical(df['Letter Grade'], categories=grade_order, ordered=True)

# Assign highest Letter Grade per ZIP code
zip_grades = df.groupby('ZIP Code')['Letter Grade'].min().reset_index()  # min picks highest grade due to ordering
zip_grades['ZIP Code'] = zip_grades['ZIP Code'].astype(str)

# Debug: Print assigned Letter Grades
print("Assigned Letter Grades per ZIP Code:")
print(zip_grades[['ZIP Code', 'Letter Grade']].sort_values('ZIP Code'))

# Merge with geodata
gdf_grades = gdf.merge(zip_grades, left_on=zip_column, right_on='ZIP Code', how='left')

# Custom colormap: F (red) to A (green)
colors = ['#FF0000', '#FF6666', '#FFFF00', '#66FF66', '#00FF00']  # F, D, C, B, A
cmap = LinearSegmentedColormap.from_list('custom_grade', colors, N=len(colors))

# Plot Letter Grade choropleth with basemap
fig, ax = plt.subplots(figsize=(12, 8))
gdf_grades.plot(column='Letter Grade', cmap=cmap, legend=True, ax=ax,
                categorical=True,
                missing_kwds={'color': 'lightgrey', 'label': 'No Data'},
                legend_kwds={'title': "Letter Grade", 'loc': 'lower left'},
                edgecolor='black', linewidth=1.5, alpha=0.7)
ctx.add_basemap(ax, crs=gdf_grades.crs, source=ctx.providers.OpenStreetMap.Mapnik)
ax.set_title('Highest School Letter Grade by ZIP Code in Atlanta Area', fontsize=14)
ax.set_axis_off()
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'zip_code_letter_grade_map.png'), dpi=300, bbox_inches='tight')
plt.close()

# --- Visualization 3: Bar Plot of Letter Grade Distribution ---
grade_counts = df['Letter Grade'].value_counts().sort_index()
fig, ax = plt.subplots(figsize=(8, 6))
sns.barplot(x=grade_counts.index, y=grade_counts.values, hue=grade_counts.index, palette='viridis', legend=False, ax=ax)
ax.set_title('Distribution of School Letter Grades', fontsize=14)
ax.set_xlabel('Letter Grade', fontsize=12)
ax.set_ylabel('Number of Schools', fontsize=12)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'letter_grade_distribution.png'), dpi=300, bbox_inches='tight')
plt.close()

# --- Visualization 4: Scatter Plot of Academic Score vs. Outcomes Score ---
fig, ax = plt.subplots(figsize=(8, 6))
sns.scatterplot(data=df, x='Academic Score', y='Outcomes Score', hue='Letter Grade',
                size='Quality Score', sizes=(20, 200), palette='coolwarm', ax=ax)
ax.set_title('Academic Score vs. Outcomes Score by Letter Grade', fontsize=14)
ax.set_xlabel('Academic Score', fontsize=12)
ax.set_ylabel('Outcomes Score', fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'academic_vs_outcomes_scatter.png'), dpi=300, bbox_inches='tight')
plt.close()

# --- Visualization 5: Box Plot of Quality Scores by ZIP Code ---
top_zips = df['ZIP Code'].value_counts().head(10).index
df_top_zips = df[df['ZIP Code'].isin(top_zips)]
fig, ax = plt.subplots(figsize=(10, 6))
sns.boxplot(x='ZIP Code', y='Quality Score', hue='ZIP Code', palette='Set2', legend=False, data=df_top_zips, ax=ax)
ax.set_title('Quality Score Distribution for Top 10 ZIP Codes', fontsize=14)
ax.set_xlabel('ZIP Code', fontsize=12)
ax.set_ylabel('Quality Score', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'quality_score_by_zip_boxplot.png'), dpi=300, bbox_inches='tight')
plt.close()

GeoJSON columns: ['STATEFP10', 'ZCTA5CE10', 'GEOID10', 'CLASSFP10', 'MTFCC10', 'FUNCSTAT10', 'ALAND10', 'AWATER10', 'INTPTLAT10', 'INTPTLON10', 'PARTFLG10', 'geometry']
Assigned Letter Grades per ZIP Code:
   ZIP Code Letter Grade
0     30002            D
1     30030            B
2     30032            B
3     30033            B
4     30034            C
5     30037            F
6     30079            D
7     30288            F
8     30303            F
9     30305            A
10    30306            D
11    30307            C
12    30308            D
13    30309            C
14    30310            C
15    30311            C
16    30312            C
17    30313            C
18    30314            C
19    30315            C
20    30316            B
21    30317            B
22    30318            C
23    30319            B
24    30324            D
25    30327            C
26    30329            C
27    30331            A
28    30337            C
29    30339            D
30    30342        